In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
%matplotlib inline
import albumentations as A
import os
import numpy as np
import pandas as pd

import albumentations as A
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings  
warnings.filterwarnings('ignore')

__print__ = print

def print(string):
    os.system(f'echo \"{string}\"')
    __print__(string)

In [ ]:
EPOCHS=20
SAMPLE_LEN=100
IMAGE_PATH="../input/plant-pathology-2021-fgvc8/train_images"
TRAIN_PATH="../input/plant-pathology-2021-fgvc8/train.csv"
SUB_PATH="../input/plant-pathology-2021-fgvc8/sample_submission.csv"
DIR_INPUT = '../input/plant-pathology-2021-fgvc8'

SEED = 42
N_FOLDS = 5
N_EPOCHS = 10
BATCH_SIZE = 64
SIZE = 224


In [ ]:
sub=pd.read_csv(SUB_PATH)
train_data=pd.read_csv(TRAIN_PATH)


In [ ]:
encoded=dict(zip(range(12),list(train_data['labels'].value_counts().keys())))

In [ ]:
encoded

In [ ]:
sub.head()

In [ ]:
class PlantTestDataset(Dataset):
    def __init__(self,df,transforms=None):
        self.image_id=df.image.values
        self.transforms=transforms
        
    def __len__(self):
        return len(self.image_id)
    def __getitem__(self,idx):
        path=DIR_INPUT+'/test_images'+'/'+self.image_id[idx]
        image=cv2.imread(path,cv2.IMREAD_COLOR)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        if self.transforms:
            image=self.transforms(image=image)['image']
            
        return image    
        

In [ ]:
class Plantmodel(nn.Module):
    
    def __init__(self,num_classes=12):
        super().__init__()
        self.model=torchvision.models.resnet18(pretrained=False)
        in_features=self.model.fc.in_features
        self.model.fc=nn.Linear(in_features,num_classes)
    def forward(self,x):
        batch_size, C, H, W = x.shape
        x=self.model(x)
       # x=self.backbone.maxpool(x)
        
       # x=self.backbone.layer1(x)
       # x=self.backbone.layer2(x)
       # x=self.backbone.layer3(x)
       # x=self.backbone.layer4(x)
        
       # x=F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
       # x=F.dropout(x,0.25,self.training)
       # x=self.classifier(x)
        return x
        

In [ ]:
transforms_train = A.Compose([
    A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
    A.Flip(),
    A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),

    # Pixels
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),

    # Affine
    A.OneOf([
        A.ElasticTransform(p=1.0),
        A.IAAPiecewiseAffine(p=1.0)
    ], p=0.5),

    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

transforms_valid = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    ToTensorV2(p=1.0),
])


In [ ]:
PATH="../input/plant-resnet/Plant_2021_epoch10.pth"
device = torch.device("cuda")
model=Plantmodel()
model.load_state_dict(torch.load(PATH))
model.to(device)

In [ ]:
test_dataset=PlantTestDataset(sub,transforms=transforms_valid)
test_loader=DataLoader(test_dataset,batch_size=64,num_workers=0,shuffle=False)

In [ ]:
prediction_all=[]
count=0
for data in test_loader:
    image=data
    image=image.to(device,dtype=torch.float)
    output=model(image)
    prediction=torch.argmax(output,dim=1)
    prediction_all.append(prediction)
    
    

In [ ]:
prediction_all=torch.cat(prediction_all,dim=0)
prediction_all=prediction_all.to('cpu').numpy()

In [ ]:
pred_label=[]
for i in prediction_all:
    pred_label.append(encoded[i])

In [ ]:
sub['labels']=pred_label

In [ ]:
sub.to_csv('submission.csv',index=False)